In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [3]:
device = torch.device('cuda:0' if  torch.cuda.is_available() else 'cpu')

In [4]:
num_epochs = 5
num_classes = 10
batch_size = 100
learnning_rate = 0.001

In [7]:
train_dataset = torchvision.datasets.MNIST(root = '../../data/',
                                          train = True,
                                          transform = transforms.ToTensor())
test_dataset = torchvision.datasets.MNIST(root = '../../data/',
                                         train = False,
                                         transform= transforms.ToTensor())

In [14]:
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = batch_size,
                                          shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                         batch_size = batch_size,
                                         shuffle = False)

In [17]:
class ConvNet(nn.Module):
    def __init__(self, num_classes = 10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [19]:
model = ConvNet(num_classes).to(device)


In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learnning_rate)

In [23]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i +1)%100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.0060
Epoch [1/5], Step [200/600], Loss: 0.0014
Epoch [1/5], Step [300/600], Loss: 0.0259
Epoch [1/5], Step [400/600], Loss: 0.0344
Epoch [1/5], Step [500/600], Loss: 0.0051
Epoch [1/5], Step [600/600], Loss: 0.0102
Epoch [2/5], Step [100/600], Loss: 0.0051
Epoch [2/5], Step [200/600], Loss: 0.0077
Epoch [2/5], Step [300/600], Loss: 0.0065
Epoch [2/5], Step [400/600], Loss: 0.0087
Epoch [2/5], Step [500/600], Loss: 0.0042
Epoch [2/5], Step [600/600], Loss: 0.0954
Epoch [3/5], Step [100/600], Loss: 0.0273
Epoch [3/5], Step [200/600], Loss: 0.0165
Epoch [3/5], Step [300/600], Loss: 0.0702
Epoch [3/5], Step [400/600], Loss: 0.0128
Epoch [3/5], Step [500/600], Loss: 0.0047
Epoch [3/5], Step [600/600], Loss: 0.0132
Epoch [4/5], Step [100/600], Loss: 0.0029
Epoch [4/5], Step [200/600], Loss: 0.0307
Epoch [4/5], Step [300/600], Loss: 0.0039
Epoch [4/5], Step [400/600], Loss: 0.0057
Epoch [4/5], Step [500/600], Loss: 0.0052
Epoch [4/5], Step [600/600], Loss:

In [25]:
model.eval()  #eval mode(batch_norm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 99.17 %


In [ ]:
torch.save(model.state_dict(), 'cn')